# Analyze Data

Korte data analyse voor het verkenne van de yelp data voor het verslag van week 1.

In [1]:
# Imports
import pandas as pd
import os
import json
import random
from IPython.display import display
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.model_selection import train_test_split
import numpy as np
import operator

In [2]:
"""
This file loads the data from the data directory and shows you how.
Feel free to change the contents of this file!
Do ensure these functions remain functional:
    - get_business(city, business_id)
    - get_reviews(city, business_id=None, user_id=None, n=10)
    - get_user(username)
"""

import os
import json
import random

DATA_DIR = "../data"


def load_cities():
    """
    Finds all cities (all directory names) in ./data
    Returns a list of city names
    """
    return os.listdir(DATA_DIR)


def load(cities, data_filename):
    """
    Given a list of city names,
        for each city extract all data from ./data/<city>/<data_filename>.json
    Returns a dictionary of the form:
        {
            <city1>: [<entry1>, <entry2>, ...],
            <city2>: [<entry1>, <entry2>, ...],
            ...
        }
    """
    data = {}
    for city in cities:
        city_data = []
        with open(f"{DATA_DIR}/{city}/{data_filename}.json", "r") as f:
            for line in f:
                city_data.append(json.loads(line))
        data[city] = city_data
    return data


def get_business(city, business_id):
    """
    Given a city name and a business id, return that business's data.
    Returns a dictionary of the form:
        {
            name:str,
            business_id:str,
            stars:str,
            ...
        }
    """
    for business in BUSINESSES[city]:
        if business["business_id"] == business_id:
            return business
    raise IndexError(f"invalid business_id {business_id}")


def get_reviews(city, business_id=None, user_id=None, n=10):
    """
    Given a city name and optionally a business id and/or auser id,
    return n reviews for that business/user combo in that city.
    Returns a dictionary of the form:
        {
            text:str,
            stars:str,
            ...
        }
    """
    def should_keep(review):
        if business_id and review["business_id"] != business_id:
            return False
        if user_id and review["user_id"] != user_id:
            return False
        return True

    reviews = REVIEWS[city]
    reviews = [review for review in reviews if should_keep(review)]
    return random.sample(reviews, min(n, len(reviews)))


def get_user(username):
    """
    Get a user by its username
    Returns a dictionary of the form:
        {
            user_id:str,
            name:str,
            ...
        }
    """
    for city, users in USERS.items():
        for user in users:
            if user["name"] == username:
                return user
    raise IndexError(f"invalid username {username}")


CITIES = load_cities()
USERS = load(CITIES, "user")
BUSINESSES = load(CITIES, "business")
REVIEWS = load(CITIES, "review")
TIPS = load(CITIES, "tip")
CHECKINS = load(CITIES, "checkin")


In [3]:
''' Schoonmaken BUSINESSES'''
#business = citymerge(BUSINESSES)
#business = business[business['is_open'] == 1 & business['categories'].notna()]
#business['categories'] = business['categories'].str.split(', ')
#print(business.latitude.mean(), business.longitude.mean())

' Schoonmaken BUSINESSES'

In [4]:
# Functie om data van alle steden samen te voegen in 1 DataFrame
def citymerge(var):
    return pd.concat([pd.DataFrame(var[city]) for city in var]).reset_index()

In [5]:
CITY = 'westlake'
reviews = pd.read_json(DATA_DIR+'/'+CITY+'/review.json', lines=True)
businesses = pd.read_json(DATA_DIR+'/'+CITY+'/business.json', lines=True)
#businesses = citymerge(BUSINESSES)
reviews_train, review_test = train_test_split(reviews)#citymerge(REVIEWS)

In [6]:
# Creeer Utility Matrix en Mean Utility Matrix uit een variabele van data.py (REVIEWS, USERS, BUSINESSES, etc.)
def create_utility_matrix(df):

    utility_matrix  = pd.pivot_table(df, index='business_id', columns='user_id', values='stars')

    mean_ultility_matrix = utility_matrix - utility_matrix.mean()
    
    return utility_matrix, mean_ultility_matrix
    
utility_matrix, mean_utility_matrix = create_utility_matrix(reviews_train)

In [7]:
def manhattan_similarity(matrix, id1, id2):
    """Compute manhattan similarity between two rows."""
    # compute distance
    distance = manhattan_distances(matrix, id1, id2)
    
    # if no distance could be computed (no shared features) return a similarity of 0
    if distance is np.nan:
        return 0
    
    # else return similarity
    return 1 / (1 + distance)

In [8]:
# Creeer Similarity Matrix uit Mean Utility Matrix
def similarity(mum):
    similarity_matrix_cosine = pd.DataFrame(cosine_similarity(mum.fillna(0)), index=mum.index, columns=mum.index).replace(0, np.nan)
    similarity_matrix_manhattan = 1 / (pd.DataFrame(manhattan_distances(mum.fillna(0)), index=mum.index, columns=mum.index).replace(0, np.nan) +1)
    similarity_matrix_euclidean = 1 / (pd.DataFrame(euclidean_distances(mum.fillna(0)), index=mum.index, columns=mum.index).replace(0, np.nan) +1)
    
    return similarity_matrix_cosine, similarity_matrix_manhattan, similarity_matrix_euclidean   
    
similarity_matrix, similarity_matrix_manhattan, similarity_matrix_euclidean = similarity(mean_utility_matrix)

In [9]:
def select_neighborhood(similarity_matrix, utility_matrix, target_user, target_business):
    """selects all items with similarity > 0"""
    # Controleer of target_user en target_business wel in de matrix zijn te vinden.
    if (target_business in similarity_matrix.index) and (target_user in utility_matrix.columns):

        # Maak een boolean mask van bedrijven die de gebruiker beoordeeld heeft met een similarity hoger dan 0.
        SelectedBusinesses = (similarity_matrix[target_business].index.isin(utility_matrix[target_user].dropna().index)) & (similarity_matrix[target_business] > 0)
    
        # return de bedrijven met de similarity door gebruik te maken van de eerder gecreeerde boolean mask.
        return similarity_matrix[target_business][SelectedBusinesses].sort_values(ascending = False)
    
    # Bij waarden die niet gevonden kunnen worden geef None terug.
    else:
        return pd.Series()

%time neighborhood = select_neighborhood(similarity_matrix, utility_matrix, 'e3fdrK1tMCwWLr76LFe-cA', 'ZTg8adZipR3QDoJmFZZqJw')
display(neighborhood)

Wall time: 2.99 ms


business_id
ZTg8adZipR3QDoJmFZZqJw    1.000000
r0DureDzsHpzs_VZem5k7g    0.077041
eigkQ_PuRON8Se265NqQDQ    0.064058
oZH3Ee7Yjk7u8B4Ed0oVOg    0.058194
0VJ8tBxOpD2OxuioVjaAxA    0.049292
irwDkp2eMP2x-4MfunRt8g    0.042602
5MWWP4Kpmw0e8d2ib9G7Kg    0.028060
spjaRNFn9Lmh4petKBuf5g    0.025428
Name: ZTg8adZipR3QDoJmFZZqJw, dtype: float64

In [10]:
def weighted_mean(neighborhood, utility_matrix, user_id):
    # Controleer of neighborhood wel een Series is en utility_matrix wel een DataFrame, anders return np.nan.
    if isinstance(neighborhood, pd.Series) and isinstance(utility_matrix, pd.DataFrame):
        # Als neighborhood of de utility_matrix leeg zijn return dan 0.
        if (neighborhood.empty) or (utility_matrix.empty):
            return np.nan
        
        # Controleer of user_id als kolom te vinden is, anders return 0.
        elif user_id in utility_matrix.columns:
    
            # Gebruik de bovenstaande formule om het gewogen gemiddelde voor de neighborhood te berekenen.
            return ((utility_matrix[user_id] * neighborhood).dropna().sum()) / (neighborhood.sum())

        else:
            return np.nan
    else:
        return np.nan
    
prediction = weighted_mean(neighborhood, utility_matrix, 'e3fdrK1tMCwWLr76LFe-cA')
print (prediction)

3.991970401743233


In [11]:
# Voorspel een score voor bedrijven en return deze als dict
def predictions(utility_matrix, similarity_matrix, user_id):
    predictdict = defaultdict()
    
    # Extract list of reviewed businesses to get new results
    review_list = utility_matrix[user_id][utility_matrix[user_id].notna()].index
    
    for business_id in similarity_matrix.index:
        if business_id in review_list:
            pass
        else:
            predictdict[business_id] = weighted_mean(select_neighborhood(similarity_matrix, utility_matrix, user_id, business_id), utility_matrix, user_id)
    
    return predictdict

In [12]:
# display(pd.read_json(DATA_DIR+'/'+CITY+'/user.json', lines=True).head())
# reviews[reviews['user_id'] == 'apP3CApEq6-z59tRLwEBYA']['business_id']

def predictions_combiner(ibcf_predictions, cb_predictions, ibcf_weight=0.8, reviews=reviews_train, businesses=businesses):
    hybrid_predictions = {}
    cb_weight = 1 - ibcf_weight
    for business_id in cb_predictions:
        if business_id in ibcf_predictions:
            ibcf_prediction = ibcf_predictions[business_id]
            cb_prediction = cb_predictions[business_id]
            hybrid_predictions[business_id] = ibcf_prediction*ibcf_weight + cb_prediction*cb_weight
            if ibcf_prediction == np.nan:
                hybrid_predictions[business_id] = cb_prediction
            if cb_prediction == np.nan:
                hybrid_predictions[business_id] = ibcf_prediction
        else:
            hybrid_predictions[business_id] = cb_prediction
            
    return  dict(sorted(hybrid_predictions.items(), key=operator.itemgetter(1), reverse=True))
    
def content_based_recommender(user_id, businesses=businesses):
    predicted_ratings = {}
    category_dict = make_category_dict(user_id)
    for business_id in businesses['business_id']:
        predicted_ratings[business_id] = content_based_predictor(user_id, business_id, category_dict)
    return predicted_ratings
    
def content_based_predictor(user_id, business_id, category_dict, reviews=reviews_train, businesses=businesses):
    ratings = []
    categories = businesses[businesses['business_id'] == business_id]['categories'].str.split(', ')
    for x in categories:
        for category in x:
            if category in category_dict:
                ratings.append(category_dict[category])
    if len(ratings) == 0:
        return np.nan
    else:
        return sum(ratings)/len(ratings)
        
def make_category_dict(user_id, reviews=reviews_train, businesses=businesses):
    category_dict = {}
    user_reviews = reviews[reviews['user_id'] == user_id]
    business_ids = user_reviews['business_id']
    for business_id in business_ids:
        categories = businesses[businesses['business_id'] == business_id]['categories'].str.split(', ')
        stars = user_reviews[user_reviews['business_id'] == business_id]['stars'].values[0]
        for x in categories:
            for category in x:
                if category in category_dict:
                    category_dict[category].append(stars)
                else:
                    category_dict[category] = [stars]
    for category in category_dict:
        category_dict[category] = sum(category_dict[category])/len(category_dict[category])
    return category_dict

%time cb_predictions = content_based_recommender('e3fdrK1tMCwWLr76LFe-cA')
%time ibcf_predictions = predictions(utility_matrix, similarity_matrix, 'e3fdrK1tMCwWLr76LFe-cA')
# cb_predictions
# ibcf_predictions
# only_icbf = predictions_combiner(ibcf_predictions, cb_predictions)
all_predictions = predictions_combiner(ibcf_predictions, cb_predictions)

Wall time: 351 ms
Wall time: 1.18 s


In [13]:
def select_items(predictions):
    pool = []
    safety = 0
    average = sum(list(predictions.values())[0:10])/10
    average -= 0.1 * average
    for prediction in predictions:
        if predictions[prediction] > average:
            pool.append(prediction)
        else:
            break
    if len(pool) < 10:
        pool = list(predictions.keys())[0:10]
    return random.sample(pool, k=10)

select_items(all_predictions)

['mW28NmePxX7pZv8lCv7v2Q',
 'C_oejk3EzfsxP7-owQDkbQ',
 'OD88wvH-9LxM_Gz4oHOIDg',
 'i7lFu1-iadoXW5Hn-JWaeg',
 '0ZN2MfHyjNIkCx7qJvVhDg',
 'nYvBZYg9rfqWFTYuxSVMdw',
 '-MsRvdPnuw6QuLn5Vxjruw',
 'KR2kRmHnRCaNzOUEGoB25w',
 'K5iqS0JXnKFFujZHIWQsag',
 'VZKWW2zQbk-rxwpAcque8w']

In [16]:
pd.Series(all_predictions).dropna()

K5iqS0JXnKFFujZHIWQsag    4.032157
i7lFu1-iadoXW5Hn-JWaeg    4.000000
nYvBZYg9rfqWFTYuxSVMdw    3.942857
0ZN2MfHyjNIkCx7qJvVhDg    3.781357
KR2kRmHnRCaNzOUEGoB25w    2.777949
_v7lMUtdd6WlgKUEBLA_VA    3.665161
UT0FUBRmpnGHE1U4Jpegxw    3.617519
2lcK3d4K7FU6O8wXdWzOmA    3.568054
QcW0360vpeEilCLBlKEiLw    3.405966
bzdb1jJ1j8Qn_RVHY97FnA    2.803260
z58nyUVyDV-vC7nXFfvR5g    3.351535
op0sZT-TNyeTMw0m3HUUDQ    4.000000
b--I9Ed6N5P9fwEb2qaiqg    3.849206
kkNWzhSpAjxm0zIxol3IzQ    3.914286
OiR6kP_mAD47vKU1WiPshg    3.957143
x5lN8HdgDVWXwoL9N7MDoQ    3.957143
7dlCzYnXDaubTAfvgAX6sQ    3.936190
mEx2NJBepbnVSjZG6MqwJA    2.904511
hTdV1_Xn1j6yhpIR7CgCJQ    3.935750
SPb3oLIVZynmiS670Vo1kQ    4.328077
95Efv0xKoUsP5lvTnaeK5w    4.037143
yG9SpxEQVIhk6e1o3jbSKg    3.857143
m9Cme6bUPuZFtqhN1AgmdQ    3.421566
5kCRty4p7tBwM9P7MAXgvA    3.391180
BsLxCm_DMxCNVEPHoaj8hw    2.877854
-mP3F3srknwKJdJ5FqcX5Q    3.474783
t10X85k_m8RuAZZgoMzvXg    3.409378
axGP6FXAZlBdmdN9juu1tQ    3.228571
EpJhRvkGDFE-GDPHM32k

In [17]:
def predict_ratings_item_based(similarity, utility, test_data):
    # For loop door alle index waarden
    for ID in test_data.index:
        # Bereken neighbourhood
        neighborhood = select_neighborhood(similarity, utility, test_data.loc[ID, 'user_id'], test_data.loc[ID, 'business_id'])
        # Voeg het gewogen gemiddelde toe als predicted rating aan de test_data Dataframe
        test_data.loc[ID, 'predicted rating'] = weighted_mean(neighborhood, utility, test_data.loc[ID, 'user_id'])
    
    return test_data
    
predicted_item_based_cosine = predict_ratings_item_based(similarity_matrix, utility_matrix, review_test[['user_id', 'business_id', 'stars']])
# predicted_item_based_manhattan = predict_ratings_item_based(similarity_matrix_manhattan, utility_matrix, review_test[['user_id', 'business_id', 'stars']])
# predicted_item_based_euclidean = predict_ratings_item_based(similarity_matrix_euclidean, utility_matrix, review_test[['user_id', 'business_id', 'stars']])

C:\Users\lverm\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\lverm\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
def mse(predicted_ratings):
    # Bereken mse uit de formule hierboven
    return ((predicted_ratings['stars'] - predicted_ratings['predicted rating']).pow(2).sum()) / (predicted_ratings.shape[0])
    
print('Cosine Similarity MSE: '+str(mse(predicted_item_based_cosine)))
# print('Manhattan Similarity MSE: '+str(mse(predicted_item_based_manhattan)))
# print('Euclidean Similarity MSE: '+str(mse(predicted_item_based_euclidean)))

Cosine Similarity MSE: 0.5969016710454935


In [21]:
test = review_test[['user_id', 'business_id', 'stars']]
test[test['user_id'] == 'e3fdrK1tMCwWLr76LFe-cA']

,user_id,business_id,stars
4258,e3fdrK1tMCwWLr76LFe-cA,uisfAopUaCje3w5_bv4nrQ,2
6818,e3fdrK1tMCwWLr76LFe-cA,oZH3Ee7Yjk7u8B4Ed0oVOg,5
5097,e3fdrK1tMCwWLr76LFe-cA,r0DureDzsHpzs_VZem5k7g,3
7054,e3fdrK1tMCwWLr76LFe-cA,6tK-R3BQ-GiMxsCGtxpJyw,1
3491,e3fdrK1tMCwWLr76LFe-cA,0VJ8tBxOpD2OxuioVjaAxA,5
2137,e3fdrK1tMCwWLr76LFe-cA,EpJhRvkGDFE-GDPHM32klw,3
7106,e3fdrK1tMCwWLr76LFe-cA,PzuyoHj3-VrYK7N8ZestNA,4
7301,e3fdrK1tMCwWLr76LFe-cA,oZH3Ee7Yjk7u8B4Ed0oVOg,5
6874,e3fdrK1tMCwWLr76LFe-cA,oZH3Ee7Yjk7u8B4Ed0oVOg,4


In [22]:
def predict_ratings_content_based(test_data):
    # For loop door alle index waarden
    total = len(test_data.index)
    current = 0
    for ID in test_data.index:
        current += 1
        if current % 100 == 0:
            print(str(current) + "\ " + str(total))
        # Bereken neighbourhood
        category_dict = make_category_dict(test_data.loc[ID, 'user_id'])
        test_data.loc[ID, 'predicted rating'] = content_based_predictor(test_data.loc[ID, 'user_id'], test_data.loc[ID, 'business_id'], category_dict)
    
    return test_data

predicted_content_based = predict_ratings_content_based(review_test[['user_id', 'business_id', 'stars']])

100\ 2224
200\ 2224
300\ 2224
400\ 2224
500\ 2224
600\ 2224
700\ 2224
800\ 2224
900\ 2224
1000\ 2224
1100\ 2224
1200\ 2224
1300\ 2224
1400\ 2224
1500\ 2224
1600\ 2224
1700\ 2224
1800\ 2224
1900\ 2224
2000\ 2224
2100\ 2224
2200\ 2224


In [23]:
%time print(mse(predicted_content_based))

0.9341032272136267
Wall time: 0 ns


In [69]:
def predictions_combiner(ibcf_predictions, cb_predictions, ibcf_weight=0.8, reviews=reviews_train, businesses=businesses):
#     hybrid_predictions = ibcf_predictions
    cb_weight = 1 - ibcf_weight
    for business_id in cb_predictions['business_id']:
        cb_prediction = cb_predictions[cb_predictions['business_id'] == business_id]['predicted rating']
        if business_id in ibcf_predictions['business_id']:
            ibcf_prediction = ibcf_predictions[ibcf_predictions['business_id'] == business_id]['predicted rating']
            hybrid_predictions[business_id] = ibcf_prediction * ibcf_weight + cb_prediction * cb_weight
            if ibcf_prediction == np.nan:
                hybrid_predictions[business_id] = cb_prediction
            if cb_prediction == np.nan:
                hybrid_predictions[business_id] = ibcf_prediction
        else:
            hybrid_predictions[business_id] = cb_prediction
    return hybrid_predictions

def df_predictions_combiner(ibcf_predictions, cb_predictions, ibcf_weight=0.8):
    hybrid_predictions = ibcf_predictions.copy()
    cb_weight = 1 - ibcf_weight
    for index, prediction in ibcf_predictions.iterrows():
        cb_prediction = cb_predictions.loc[index]['predicted rating']
        ibcf_prediction = prediction['predicted rating']
        print(cb_prediction)
        print(ibcf_prediction)
        weighted_average = cb_prediction * cb_weight + ibcf_prediction * ibcf_weight
        hybrid_predictions.loc[index, hybrid_predictions.columns.get_loc('predicted rating')] = weighted_average
        if ibcf_prediction == np.nan:
            hybrid_predictions.loc[index, hybrid_predictions.columns.get_loc('predicted rating')] = cb_prediction
        if cb_prediction == np.nan:
            hybrid_predictions.loc[index, hybrid_predictions.columns.get_loc('predicted rating')] = ibcf_prediction
        print(hybrid_predictions.loc[index, hybrid_predictions.columns.get_loc('predicted rating')])
        print('-------------')
    return hybrid_predictions
        
    
# %time hybrid_predictions = df_predictions_combiner(predicted_item_based_cosine, predicted_content_based, 0.8)


nan

In [70]:
def calculate_mse_weights(predicted_item_based_cosine, predicted_content_based):
    for weight in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        hybrid_predictions = df_predictions_combiner(predicted_item_based_cosine, predicted_content_based, weight)
        mean_square_error = mse(hybrid_predictions)
        print("MSE for weight: " + str(weight) + " is " + str(mean_square_error))

calculate_mse_weights(predicted_item_based_cosine, predicted_content_based)

nan
nan
nan
-------------
3.5
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.875
5.0
3.9875000000000003
-------------
3.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
3.722222222222222
4.664128405470637
3.8164128405470636
-------------
2.3333333333333335
2.0
2.3000000000000003
-------------
1.0
1.0
1.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
2.4255759199626326
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
4.0
nan
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
nan
nan
nan
-------------
2.0
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
2.6
3.4762088546940317
2.6876208854694035
-------------
4.55555

4.25
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
3.96875
4.122223962913621
3.984097396291362
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.45
5.0
4.505
-------------
nan
nan
nan
-------------
4.0
4.0
4.0
-------------
4.166666666666666
4.0
4.1499999999999995
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
3.3333333333333335
4.0
3.4
-------------
4.0
nan
nan
-------------
3.75
nan
nan
-------------
5.0
nan
nan
-------------
4.5
3.0
4.35
-------------
nan
nan
nan
-------------
3.5
4.0


nan
-------------
2.0
nan
nan
-------------
3.444444444444444
4.0
3.4999999999999996
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.0
3.0
3.0
-------------
3.0
3.1317132074813148
3.013171320748132
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
4.316666666666666
4.822874385987791
4.367287438598779
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
2.75
3.673012200805145
2.8423012200805147
-------------
4.083333333333333
3.0
3.9749999999999996
-------------
4.5
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.0
4.444233206064412
3.1444233206064416
-------------
3.0083333333333333
2.215217590137298
2.9290217590137297
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
----------

nan
-------------
nan
nan
nan
-------------
2.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
4.0
4.0
-------------
nan
nan
nan
-------------
1.8
nan
nan
-------------
2.5
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
4.0
4.0
-------------
nan
nan
nan
-------------
5.0
4.999999999999999
5.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
3.6
4.0
3.64
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.0
4.959822248594899
3.19598222485949
-------------
nan
nan
nan
-------------
3.3666666666666667
2.407403418592601
3.2707403418592604
-------------
nan
5.0
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.333333333333333
nan
nan
-------------
4.45
5.0
4.505
-------------
3.5595238095238093
4.

nan
nan
nan
-------------
2.5
nan
nan
-------------
3.1666666666666665
2.0
3.0500000000000003
-------------
nan
nan
nan
-------------
nan
4.825037605400704
nan
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
2.3333333333333335
2.0
2.3000000000000003
-------------
1.0
1.0
1.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
3.5
nan
nan
-------------
3.3333333333333335
3.225136695473216
3.3225136695473214
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.0
3.0
3.0
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
2.277777777777778
3.8439659781608597
2.4343965978160864
-------------
5.0
4.494694467173563
4.94946944

...

1.0
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
4.0
4.0
4.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
2.7285848460990536
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.0
3.0
3.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
2.7
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
4.0
4.0
4.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.0
3.0
3.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.5714285714285716
3.3138224904733633
3.3653437066644054
-------------
nan
nan
nan


nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
1.0
1.0
1.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
1.8333333333333333
2.0
1.9666666666666668
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
1.5
1.0
1.1
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
3.8421052631578947
4.63268746977422
4.4745710284509554
-------------
4.1
4.0
4.02
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
3.75
4.0
3.95
-------------
nan
nan
nan
-------------
3.75
4.155228116452739
4.0741824931621915
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
4.0
4.010646694374812
4.008517355499849
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
3.4166666666666665
4.0
3.8833333333333333
-------------
3.35
4.4283373876654455
4.212669910132

nan
-------------
4.0
nan
nan
-------------
4.0
3.2614238853172672
3.4091391082538136
-------------
nan
nan
nan
-------------
2.9027777777777772
5.0
4.580555555555556
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
nan
nan
nan
-------------
2.84375
2.0
2.16875
-------------
5.0
nan
nan
-------------
3.0
nan
nan
-------------
nan
nan
nan
-------------
3.333333333333333
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.333333333333333
5.0
4.866666666666666
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
1.0
1.0
1.0
-------------
nan
nan
nan
-------------
nan
nan
nan
---

nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
5.0
5.0
5.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.375
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
1.0
1.0
1.0
-------------
3.9166666666666665
nan
nan
-------------
4.0
4.0
4.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
4.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
-------------
5.0
5.0
5.0
-------------
2.375
nan
nan
-------------
4.0
nan
nan
-------------
nan
nan
nan
-------------
5.0
nan
nan
-------------
nan
nan
nan
-------------
3.05
nan
nan
-------------
5.0
nan
nan
-------------
5.0
5.000000000000001
5.000000000000001
-------------
4.0
nan
nan
-------------
nan
nan
nan
-------------
2.0
nan
nan
-------------
nan
nan
nan
-------------
nan
nan
nan
----

KeyboardInterrupt: 